# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
DF = "Weatherdata.csv"
Data = pd.read_csv(DF)
Data.dropna(inplace = True)
Data.head()

,City,Latitude,Longitude,Maximum Temperature (F),Humidity,Cloudiness,Wind Speed,Country,Date and Time
0,Bethel,41.3712,-73.4140,22.05,68,0,0.00,US,2022-01-31 04:31:56
1,Rikitea,-23.1203,-134.9692,77.38,64,100,5.79,PF,2022-01-31 04:31:56
2,Mataura,-46.1927,168.8643,73.87,57,92,4.94,NZ,2022-01-31 04:31:57
3,Nikolskoye,59.7035,30.7861,26.49,96,100,5.06,RU,2022-01-31 04:31:57
4,Mindelo,16.8901,-24.9804,69.35,60,75,20.71,CV,2022-01-31 04:31:57


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(g_key)
#Gather coordinates and humidity
Coordinates = Data[["Latitude","Longitude"]]
Humidity = Data["Humidity"].astype(float)

In [ ]:
# Customize the size of the figure
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure(layout=figure_layout)

# Create heat layer
Hum_layer = gmaps.heatmap_layer(Coordinates, weights=Humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(Hum_layer)

# Display figure
fig

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [15]:
Conditions = Data.loc[(Data["Maximum Temperature (F)"] < 65) & 
                      (Data["Maximum Temperature (F)"] > 35) & (Data["Cloudiness"] > 20) & (Data["Wind Speed"] < 10)
                     & (Data["Humidity"] < 60)]
Conditions = Conditions.dropna()
Conditions.head(10)

,City,Latitude,Longitude,Maximum Temperature (F),Humidity,Cloudiness,Wind Speed,Country,Date and Time,Hotel Name
46,Rawson,-43.3002,-65.1023,58.05,33,40,9.35,AR,2022-01-31 04:32:09,Hosteria Sampedro
74,Laas,46.6166,10.7002,42.26,46,66,4.79,IT,2022-01-31 04:32:17,Pension Feldgärtenhof
112,Arrondissement de Nevers,46.9167,3.3333,41.59,1,67,7.78,FR,2022-01-31 04:30:02,Castle Prye
166,Hasaki,35.7333,140.8333,50.29,28,29,6.31,JP,2022-01-31 04:32:43,Hotel Sunrise Choshi
200,Panzhihua,26.5851,101.7128,61.45,44,48,1.07,CN,2022-01-31 04:32:52,Electric Power Hotel
278,Levelland,33.5873,-102.3780,52.95,51,100,4.05,US,2022-01-31 04:33:15,"Holiday Inn Express & Suites Levelland, an IHG..."
300,Jaen,38.0000,-3.5000,45.93,37,96,1.83,ES,2022-01-31 04:33:21,Hotel TRH Baeza
338,Lata,40.1629,-8.3327,39.06,52,22,3.65,PT,2022-01-31 04:33:32,Casa Maquia
341,Pala,9.3642,14.9046,59.32,25,23,5.84,TD,2022-01-31 04:33:33,Nectar Hôtel de Pala
360,Nara,34.6851,135.8049,47.97,50,56,8.05,JP,2022-01-31 04:33:38,Guesthouse Nara Komachi


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [24]:
Hotel_df = Data

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}


for index, row in Hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    # print(json.dumps(response, indent=4, sort_keys=True))
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        Hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("No hotel noted.")


Retrieving Results for Index 0: Bethel.
Closest hotel is Courtyard by Marriott Danbury.
Retrieving Results for Index 1: Rikitea.
Closest hotel is People ThankYou.
Retrieving Results for Index 2: Mataura.
Closest hotel is Ellie's Villa.
Retrieving Results for Index 3: Nikolskoye.
Closest hotel is Tourist House - Sablino.
Retrieving Results for Index 4: Mindelo.
Closest hotel is Hotel Gaudi.
Retrieving Results for Index 5: Nanortalik.
Closest hotel is Hotel Kap Farvel.
Retrieving Results for Index 6: Atuona.
Closest hotel is Villa Enata.
Retrieving Results for Index 7: Sabang.
Closest hotel is Hotel Citra.
Retrieving Results for Index 8: Tasiilaq.
Closest hotel is The Red House.
Retrieving Results for Index 9: Hermanus.
Closest hotel is Misty Waves Boutique Hotel.
Retrieving Results for Index 10: Vaini.
Closest hotel is Keleti Beach Resort.
Retrieving Results for Index 11: New Norfolk.
Closest hotel is The Shingles Riverside Cottages.
Retrieving Results for Index 12: Ribeira Grande.
Clos

In [17]:
Hotel_df.head(10)


,City,Latitude,Longitude,Maximum Temperature (F),Humidity,Cloudiness,Wind Speed,Country,Date and Time,Hotel Name
46,Rawson,-43.3002,-65.1023,58.05,33,40,9.35,AR,2022-01-31 04:32:09,Hosteria Sampedro
74,Laas,46.6166,10.7002,42.26,46,66,4.79,IT,2022-01-31 04:32:17,Pension Feldgärtenhof
112,Arrondissement de Nevers,46.9167,3.3333,41.59,1,67,7.78,FR,2022-01-31 04:30:02,Castle Prye
166,Hasaki,35.7333,140.8333,50.29,28,29,6.31,JP,2022-01-31 04:32:43,Hotel Sunrise Choshi
200,Panzhihua,26.5851,101.7128,61.45,44,48,1.07,CN,2022-01-31 04:32:52,Electric Power Hotel
278,Levelland,33.5873,-102.3780,52.95,51,100,4.05,US,2022-01-31 04:33:15,"Holiday Inn Express & Suites Levelland, an IHG..."
300,Jaen,38.0000,-3.5000,45.93,37,96,1.83,ES,2022-01-31 04:33:21,Hotel TRH Baeza
338,Lata,40.1629,-8.3327,39.06,52,22,3.65,PT,2022-01-31 04:33:32,Casa Maquia
341,Pala,9.3642,14.9046,59.32,25,23,5.84,TD,2022-01-31 04:33:33,Nectar Hôtel de Pala
360,Nara,34.6851,135.8049,47.97,50,56,8.05,JP,2022-01-31 04:33:38,Guesthouse Nara Komachi


In [33]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in Hotel_df.iterrows()]
# NOTE: be sure to update with your DataFrame name
Locations = Hotel_df[["Latitude", "Longitude"]]

TypeError: 'DataFrame' object is not callable

In [31]:
# Add marker layer ontop of heat map
# Assign the marker layer to a variable
markers = gmaps.marker_layer(Locations)
# Add the layer to the map
fig.add_layer(markers)
#fig.add_layer(hotel_layer)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))